In [1]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 869.1/869.1 kB 13.1 MB/s eta 0:00:00a 0:00:01


In [2]:
import os
import shutil
import numpy as np
import yaml
from ultralytics import YOLO

## Prepare dataset

In [3]:
DATA_CURR_DIR = "/kaggle/input/face-detection-dataset"
DATA_DEST_DIR = "/kaggle/working/face-detection-dataset"
DATA_SPLITS_NAMES = ("train", "val", "test")
TEST_SPLIT_FRACTION = 0.25

In [7]:
def convert_to_yolo(split_name: str) -> None:
    """
      Convert dataset to ultralytics yolo format.

      Args:
        split_name (str): Name of the split (train/val/test)

      Returns:
        None: Due to OS actions
    """ 
    # Create dirs.
    split_dir_images = os.path.join(DATA_DEST_DIR, split_name, "images")
    split_dir_labels = os.path.join(DATA_DEST_DIR, split_name, "labels")
    if not os.path.exists(split_dir_images): os.makedirs(split_dir_images) 
    if not os.path.exists(split_dir_labels): os.makedirs(split_dir_labels)

    # Fill dirs /images, /labels
    if split_name != "test":
        curr_dir_names = ("images", "labels")
        for curr_dir_name in curr_dir_names:
            files_path = os.path.join(DATA_CURR_DIR, curr_dir_name, split_name)
            if curr_dir_name == "images": dest_dir_name = split_dir_images
            else: dest_dir_name = split_dir_labels
            for path in os.listdir(files_path):
                file_path = os.path.join(files_path, path)
                if os.path.isfile(file_path):
                    shutil.copy(file_path, dest_dir_name)

In [5]:
for split_name in DATA_SPLITS_NAMES:
    print("Processing split:", split_name)
    convert_to_yolo(split_name)

Processing split: train
Processing split: val
Processing split: test


### Create split for test.

In [6]:
# Get filenames from /train for /test with corresponds of test fraction (TEST_SPLIT_FRACTION).

train_image_names = [file_name for file_name in os.scandir(os.path.join(DATA_DEST_DIR, "train", "images"))]
test_split_count = int(len(train_image_names) * TEST_SPLIT_FRACTION)
idxs = np.random.randint(0, high=len(train_image_names), size=test_split_count, dtype=int)

for idx in idxs:
    # Image.
    image_filename = train_image_names[idx].name
    image_train_path = os.path.join(DATA_DEST_DIR, "train", "images", image_filename)
    image_test_path = os.path.join(DATA_DEST_DIR, "test", "images", image_filename)

    # Label.
    label_filename = train_image_names[idx].name.split(".")[0] + ".txt"
    label_train_path = os.path.join(DATA_DEST_DIR, "train", "labels", label_filename)
    label_test_path = os.path.join(DATA_DEST_DIR, "test", "labels", label_filename)

    if os.path.exists(image_train_path) and os.path.exists(label_train_path):
        shutil.move(image_train_path, image_test_path, copy_function=shutil.copy)        
        shutil.move(label_train_path, label_test_path, copy_function=shutil.copy)
    else: print("Doesn't exist one of this:", image_train_path, label_train_path)

Doesn't exist one of this: /kaggle/working/face-detection-dataset/train/images/a0ae1a3f1f67577b.jpg /kaggle/working/face-detection-dataset/train/labels/a0ae1a3f1f67577b.txt
Doesn't exist one of this: /kaggle/working/face-detection-dataset/train/images/087d8489f86303d8.jpg /kaggle/working/face-detection-dataset/train/labels/087d8489f86303d8.txt
Doesn't exist one of this: /kaggle/working/face-detection-dataset/train/images/ba7e549961d07089.jpg /kaggle/working/face-detection-dataset/train/labels/ba7e549961d07089.txt
Doesn't exist one of this: /kaggle/working/face-detection-dataset/train/images/3f133b2e7158cf57.jpg /kaggle/working/face-detection-dataset/train/labels/3f133b2e7158cf57.txt
Doesn't exist one of this: /kaggle/working/face-detection-dataset/train/images/5bb75d11536a0829.jpg /kaggle/working/face-detection-dataset/train/labels/5bb75d11536a0829.txt
Doesn't exist one of this: /kaggle/working/face-detection-dataset/train/images/a8cd36fbc1d0e7b9.jpg /kaggle/working/face-detection-data

### Check if everything is ok with test split.

In [4]:
test_image_names = [file_name.name.split(".")[0] for file_name in os.scandir(os.path.join(DATA_DEST_DIR, "test", "images"))]
test_label_names = [file_name.name.split(".")[0] for file_name in os.scandir(os.path.join(DATA_DEST_DIR, "test", "labels"))]

In [5]:
len(test_image_names), len(test_label_names), len(test_image_names) == len(test_label_names)

(2939, 2939, True)

In [6]:
# Convert lists to sets
set1 = set(test_image_names)
set2 = set(test_label_names)

# Difference (elements in set1 not in set2)
difference = set1 - set2
print(list(difference))

[]


### Create data.yaml

In [10]:
dataset_config = {
    "names": ["face"],
    "nc": 1,  
    "train": os.path.join(DATA_DEST_DIR, "train/images"),  
    "val": os.path.join(DATA_DEST_DIR, "val/images"),
    "test": os.path.join(DATA_DEST_DIR, "test/images")
}

with open("data.yaml", "w") as file:
    yaml.dump(dataset_config, file, default_flow_style=False)

## Train model.

In [3]:
# Loading the YOLOv8 model
# model = YOLO("yolov8m.pt", task="detect") 
model = YOLO("/kaggle/working/face_detection/yolov8m_face_detection_s640_e30-50/weights/yolov8m_face_detection_e50_s640_best.pt", 
             task="detect") # Past epochs: 50


# Setting up training arguments
args = dict(
  data="data.yaml", 
  epochs=20,
  batch=-1,
  imgsz=640,
  cache=True,
  device=0,
  project="face_detection",
  name="yolov8m_face_detection_s640_e50-70",
  verbose=True,
#   cos_lr=True, # Turned off for learning from 50 to 70 epochs.
  plots=True
)

# Initiating model training
results = model.train(**args)


Ultralytics YOLOv8.2.79 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: task=detect, mode=train, model=/kaggle/working/face_detection/yolov8m_face_detection_s640_e30-50/weights/yolov8m_face_detection_e50_s640_best.pt, data=data.yaml, epochs=20, time=None, patience=100, batch=-1, imgsz=640, save=True, save_period=-1, cache=True, device=0, workers=8, project=face_detection, name=yolov8m_face_detection_s640_e50-703, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embe

100%|██████████| 755k/755k [00:00<00:00, 14.0MB/s]
2024-08-22 07:43:19,641	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-08-22 07:43:21,037	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.



                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  ultralytics.nn.modules.conv.Conv             [3, 48, 3, 2]                 
  1                  -1  1     41664  ultralytics.nn.modules.conv.Conv             [48, 96, 3, 2]                
  2                  -1  2    111360  ultralytics.nn.modules.block.C2f             [96, 96, 2, True]             
  3                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  4                  -1  4    813312  ultralytics.nn.modules.block.C2f             [192, 192, 4, True]           
  5                  -1  1    664320  ultralytics.nn.modules.conv.Conv             [192, 384, 3, 2]              
  6                  -1  4   3248640  ultralytics.nn.modules.block.C2f             [384, 384, 4, True]           
  7                  -1  1   1991808  ultralytics.nn.modules.conv.Conv             [384

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


...

                   all       3347      10299      0.896      0.821       0.89      0.595



20 epochs completed in 2.471 hours.
Optimizer stripped from face_detection/yolov8m_face_detection_s640_e50-703/weights/last.pt, 52.0MB
Optimizer stripped from face_detection/yolov8m_face_detection_s640_e50-703/weights/best.pt, 52.0MB

Validating face_detection/yolov8m_face_detection_s640_e50-703/weights/best.pt...
Ultralytics YOLOv8.2.79 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
Model summary (fused): 218 layers, 25,840,339 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 77/77 [00:43<00:00,  1.75it/s]


                   all       3347      10299      0.895      0.817      0.888      0.595
Speed: 0.1ms preprocess, 7.8ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to face_detection/yolov8m_face_detection_s640_e50-703


lr/pg0,▃▆██▇▇▆▆▆▅▅▄▄▃▃▃▂▂▁▁
lr/pg1,▃▆██▇▇▆▆▆▅▅▄▄▃▃▃▂▂▁▁
lr/pg2,▃▆██▇▇▆▆▆▅▅▄▄▃▃▃▂▂▁▁
metrics/mAP50(B),▆▄▂▂▂▁▃▃▃▃▄▅▆▇▇█████
metrics/mAP50-95(B),▆▄▃▃▃▁▃▃▄▃▅▆▆▇▇▇▇███
metrics/precision(B),▄▃▆▄▅▁▆▅▄▅▄▆█▆█▇██▆█
metrics/recall(B),▇▇▁▄▃▃▁▄▄▄▅▄▄▇▆█▆▇█▇
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,▂▁▂▂▂▁▁▁▁▁██████▇▇▇▇


## Validate on test split.

In [5]:
# Check metrics on test split.
model = YOLO("/kaggle/working/face_detection/yolov8m_face_detection_s640_e30-50/weights/yolov8m_face_detection_e50_s640_best.pt", 
             task="detect"
) # because "_e50_" model slightly better.

metrics = model.val(data="data.yaml", device=0, split="test")

Ultralytics YOLOv8.2.79 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
Model summary (fused): 218 layers, 25,840,339 parameters, 0 gradients, 78.7 GFLOPs


val: Scanning /kaggle/working/face-detection-dataset/test/labels.cache... 2939 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2939/2939 [00:00<?, ?it/s]
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 184/184 [00:40<00:00,  4.50it/s]


                   all       2939       8826        0.9       0.82      0.884      0.595
Speed: 0.1ms preprocess, 8.0ms inference, 0.0ms loss, 1.5ms postprocess per image
Results saved to runs/detect/val2


In [9]:
print("mAP50-95", metrics.box.map)
print("mAP50", metrics.box.map50)
print("mAP75", metrics.box.map75)
print("mAP50-95", metrics.box.maps[0])

mAP50-95 0.5951788936471754
mAP50 0.8841639137917188
mAP75 0.6730644622409729
mAP50-95 0.5951788936471754
